### Camera

In [1]:
# Step 1: Clean slate
from jetracer.camera_utils import JetRacerCamera, release_cam
import time

# Step 2: Release any existing cameras
release_cam()
time.sleep(2)

# Step 3: Create camera for your use case
camera = JetRacerCamera('training')  # or 'inference' or 'safe'

# Step 4: Start camera with error handling
if camera.start():
    print("✅ Camera initialized successfully!")
    print(f"Resolution: {camera.width}x{camera.height}")
    print(f"Mode: {camera.mode}")
    
    # Test image capture
    img = camera.read()
    if img is not None:
        print(f"✅ Capturing images: {img.shape}")
        print("🎯 Camera ready for use!")
    else:
        print("❌ Camera not capturing images")
else:
    print("❌ Camera failed to initialize")
    print("🔧 Try restarting Jupyter kernel")

🔄 Releasing all cameras...
Found camera objects: ['CSICamera', 'JetRacerCamera']
✓ Stopped CSICamera.running
✓ Deleted CSICamera
✓ Stopped JetRacerCamera.running
Warning cleaning JetRacerCamera: JetRacerCamera.stop() missing 1 required positional argument: 'self'
✓ Garbage collection completed
⏳ Waiting for camera hardware to release...
✅ Camera release complete!


RuntimeError: Could not initialize camera.  Please see error trace.

In [2]:
import cv2
import time

# lets actually see what the camera is capturing
cap = cv2.VideoCapture(0)

if cap.isOpened():
    print("camera opened")
    
    # capture a few frames and save them to see what we're getting
    for i in range(3):
        ret, frame = cap.read()
        if ret:
            filename = f"test_frame_{i}.jpg"
            cv2.imwrite(filename, frame)
            print(f"saved {filename} - shape: {frame.shape}")
            
            # show some pixel values to see if its actual image data
            print(f"top left corner pixels: {frame[0:3, 0:3, 0]}")
            
        time.sleep(1)
    
    cap.release()
    print("done - check the saved images")
else:
    print("camera failed to open")

camera failed to open


[ WARN:0@38.609] global cap_v4l.cpp:914 open VIDEOIO(V4L2:/dev/video0): can't open camera by index
[ WARN:0@38.610] global obsensor_stream_channel_v4l2.cpp:82 xioctl ioctl: fd=57, req=-2140645888
[ WARN:0@38.610] global obsensor_stream_channel_v4l2.cpp:138 queryUvcDeviceInfoList ioctl error return: 25
[ WARN:0@38.610] global obsensor_stream_channel_v4l2.cpp:82 xioctl ioctl: fd=57, req=-2140645888
[ WARN:0@38.610] global obsensor_stream_channel_v4l2.cpp:138 queryUvcDeviceInfoList ioctl error return: 25
[ WARN:0@38.610] global obsensor_stream_channel_v4l2.cpp:82 xioctl ioctl: fd=57, req=-2140645888
[ WARN:0@38.610] global obsensor_stream_channel_v4l2.cpp:138 queryUvcDeviceInfoList ioctl error return: 25
[ERROR:0@38.610] global obsensor_uvc_stream_channel.cpp:163 getStreamChannelGroup Camera index out of range


In [1]:
# camera.py - basic camera that actually works
import cv2
import time
import numpy as np

class AutonomousRacecarCamera:
    def __init__(self, mode='inference', width=640, height=480, fps=21):
        self.mode = mode
        self.width = width  
        self.height = height
        self.fps = fps
        
        # figure out what size to output
        if mode == 'inference' or mode == 'training':
            self.output_size = (224, 224)  # ai model size
        else:
            self.output_size = (width, height)  # debug size
            
        self.camera = None
        self.running = False
        self.last_frame = None
        
    def start(self):
        print(f"starting camera in {self.mode} mode")
        
        try:
            # just use device 0 since thats what works
            self.camera = cv2.VideoCapture(0)
            
            if self.camera.isOpened():
                # test it
                ret, frame = self.camera.read()
                if ret and frame is not None:
                    print(f"camera works - native size: {frame.shape}")
                    self.running = True
                    return True
                else:
                    print("camera opened but no image")
                    return False
            else:
                print("camera wont open")
                return False
                
        except Exception as e:
            print(f"camera failed: {e}")
            return False
    
    def read(self):
        if not self.running or not self.camera:
            return self.last_frame
            
        try:
            ret, frame = self.camera.read()
            if ret and frame is not None:
                # resize to what we need
                resized = cv2.resize(frame, self.output_size)
                self.last_frame = resized
                return resized
            else:
                return self.last_frame
        except Exception as e:
            print(f"read failed: {e}")
            return self.last_frame
    
    def stop(self):
        self.running = False
        if self.camera:
            self.camera.release()
        print("camera stopped")
    
    @property
    def value(self):
        # for compatibility with old code
        return self.read()
    
    def __enter__(self):
        self.start()
        return self
    
    def __exit__(self, *args):
        self.stop()

# easy functions
def create_inference_camera():
    return AutonomousRacecarCamera('inference')

def create_training_camera():
    return AutonomousRacecarCamera('training')

def create_debug_camera():
    return AutonomousRacecarCamera('debug')

def test_camera():
    print("testing camera")
    
    cam = AutonomousRacecarCamera('debug')
    if cam.start():
        for i in range(3):
            img = cam.read()
            if img is not None:
                print(f"frame {i}: {img.shape}")
            time.sleep(0.5)
        cam.stop()
        return True
    else:
        print("camera test failed")
        return False

if __name__ == "__main__":
    test_camera()

testing camera
starting camera in debug mode
camera wont open
camera test failed


[ WARN:0@0.020] global cap_v4l.cpp:914 open VIDEOIO(V4L2:/dev/video0): can't open camera by index
[ WARN:0@0.021] global obsensor_stream_channel_v4l2.cpp:82 xioctl ioctl: fd=57, req=-2140645888
[ WARN:0@0.021] global obsensor_stream_channel_v4l2.cpp:138 queryUvcDeviceInfoList ioctl error return: 25
[ WARN:0@0.021] global obsensor_stream_channel_v4l2.cpp:82 xioctl ioctl: fd=57, req=-2140645888
[ WARN:0@0.021] global obsensor_stream_channel_v4l2.cpp:138 queryUvcDeviceInfoList ioctl error return: 25
[ WARN:0@0.021] global obsensor_stream_channel_v4l2.cpp:82 xioctl ioctl: fd=57, req=-2140645888
[ WARN:0@0.021] global obsensor_stream_channel_v4l2.cpp:138 queryUvcDeviceInfoList ioctl error return: 25
[ERROR:0@0.021] global obsensor_uvc_stream_channel.cpp:163 getStreamChannelGroup Camera index out of range


In [2]:
# Get the current frame from camera
image = camera.value

# Display image info
print(f"Image shape: {image.shape}")
print(f"Image type: {type(image)}")

NameError: name 'camera' is not defined

### Task

### Data Collection

In [3]:
# 📸 IMPROVED DATA COLLECTION STRATEGY
# Better training images to fix steering issues

import cv2
import ipywidgets
import threading
import time
import os
import uuid
from IPython.display import display, HTML

print("📸 IMPROVED DATA COLLECTION FOR BETTER TRAINING")
print("=" * 55)

# Clear old data first
def clear_old_training_data():
    """Clear old training data"""
    print("🗑️ CLEARING OLD TRAINING DATA")
    print("=" * 35)
    
    import shutil
    
    old_dirs = [
        "/home/checker/Autonomous-Racecar/data",
        "/home/checker/Autonomous-Racecar/training_data"
    ]
    
    for old_dir in old_dirs:
        if os.path.exists(old_dir):
            try:
                shutil.rmtree(old_dir)
                print(f"✅ Cleared: {old_dir}")
            except Exception as e:
                print(f"⚠️ Error clearing {old_dir}: {e}")
    
    print("✅ Old data cleared - ready for fresh collection")

# Improved data collection categories
class ImprovedDataCollection:
    """Enhanced data collection with specific driving scenarios"""
    
    def __init__(self):
        self.base_dir = "/home/checker/Autonomous-Racecar/data/improved_training"
        self.collecting = False
        
        # IMPROVED CATEGORIES - More specific scenarios
        self.categories = {
            'straight_center': "Going straight down center of track",
            'straight_left_correct': "Correcting from left side back to center", 
            'straight_right_correct': "Correcting from right side back to center",
            'left_turn_gentle': "Gentle left turn entry",
            'left_turn_sharp': "Sharp left turn", 
            'left_turn_exit': "Exiting left turn back to straight",
            'recovery_left': "Too far left - need to correct right",
            'recovery_right': "Too far right - need to correct left"
        }
        
        # Create directories
        for category in self.categories.keys():
            os.makedirs(f"{self.base_dir}/{category}", exist_ok=True)
        
        self.counts = {cat: 0 for cat in self.categories.keys()}
        
        print("📁 Created improved categories:")
        for cat, desc in self.categories.items():
            print(f"   {cat}: {desc}")

def setup_improved_camera():
    """Setup camera using your working method"""
    print("\n📷 Setting up camera...")
    try:
        from jetcam.csi_camera import CSICamera
        camera = CSICamera(width=224, height=224, capture_fps=21)
        camera.running = True
        time.sleep(3)
        print("✅ Camera ready")
        return camera
    except Exception as e:
        print(f"❌ Camera failed: {e}")
        return None

def create_improved_collection_interface():
    """Create improved data collection interface"""
    print("\n🎛️ CREATING IMPROVED COLLECTION INTERFACE")
    print("=" * 50)
    
    # Initialize camera
    camera = setup_improved_camera()
    if camera is None:
        return
    
    # Initialize data collector
    collector = ImprovedDataCollection()
    
    # Image display
    image_widget = ipywidgets.Image(format='jpeg', width=400, height=400)
    
    # Category selection dropdown
    category_widget = ipywidgets.Dropdown(
        options=list(collector.categories.keys()),
        value='straight_center',
        description='Scenario:'
    )
    
    # Count displays for each category
    count_displays = {}
    for category in collector.categories.keys():
        count_displays[category] = ipywidgets.IntText(
            value=0, 
            description=f'{category}:',
            disabled=True,
            layout=ipywidgets.Layout(width='200px')
        )
    
    # Quick capture buttons for common scenarios
    buttons = {}
    button_configs = [
        ('straight_center', "⬆️ STRAIGHT CENTER", 'success'),
        ('straight_left_correct', "↗️ CORRECT→CENTER", 'info'),
        ('straight_right_correct', "↖️ CORRECT→CENTER", 'info'),
        ('left_turn_gentle', "↩️ LEFT GENTLE", 'warning'),
        ('left_turn_sharp', "↩️ LEFT SHARP", 'danger'),
        ('left_turn_exit', "↪️ EXIT LEFT TURN", 'primary'),
        ('recovery_left', "🔄 TOO LEFT→RIGHT", 'danger'),
        ('recovery_right', "🔄 TOO RIGHT→LEFT", 'danger')
    ]
    
    for category, label, style in button_configs:
        buttons[category] = ipywidgets.Button(
            description=label,
            button_style=style,
            layout=ipywidgets.Layout(width='180px', height='40px')
        )
    
    # Control buttons
    start_button = ipywidgets.Button(description="📷 Start Collection", button_style='primary')
    stop_button = ipywidgets.Button(description="⏹️ Stop Collection", button_style='danger', disabled=True)
    
    # Status display
    status_widget = ipywidgets.HTML(value="<b>Ready for improved data collection</b>")
    
    # Instructions
    instructions = ipywidgets.HTML(value="""
    <h3>📋 IMPROVED DATA COLLECTION INSTRUCTIONS:</h3>
    <b>🎯 DRIVING SCENARIOS TO COLLECT:</b><br>
    <b>STRAIGHTS:</b><br>
    • straight_center: Perfect center-line driving<br>
    • straight_left_correct: Car slightly left, needs to correct right<br>
    • straight_right_correct: Car slightly right, needs to correct left<br><br>
    <b>LEFT TURNS:</b><br>
    • left_turn_gentle: Easy left curves<br>
    • left_turn_sharp: Tight left turns<br>
    • left_turn_exit: Coming out of turn, straightening<br><br>
    <b>RECOVERY:</b><br>
    • recovery_left: Car too far left, needs correction<br>
    • recovery_right: Car too far right, needs correction<br><br>
    <b>🎯 COLLECTION STRATEGY:</b><br>
    1. Position car in specific scenario<br>
    2. Click appropriate button for what car should do<br>
    3. Collect 30-50 examples per scenario<br>
    4. Focus on RECOVERY scenarios (fixes oversteer issue!)
    """)
    
    def update_counts():
        """Update count displays"""
        for category in collector.categories.keys():
            path = f"{collector.base_dir}/{category}"
            if os.path.exists(path):
                count = len([f for f in os.listdir(path) if f.endswith('.jpg')])
                collector.counts[category] = count
                count_displays[category].value = count
    
    def save_image(category):
        """Save image with category"""
        try:
            img = camera.value
            if img is None:
                return False
            
            # Generate filename
            timestamp = int(time.time() * 1000)
            filename = f"{category}_{timestamp}_{str(uuid.uuid4())[:8]}.jpg"
            filepath = f"{collector.base_dir}/{category}/{filename}"
            
            # Save image
            cv2.imwrite(filepath, img)
            
            # Update count
            collector.counts[category] += 1
            count_displays[category].value = collector.counts[category]
            
            # Update status
            total = sum(collector.counts.values())
            status_widget.value = f"<b>✅ Saved {category}: #{collector.counts[category]} (Total: {total})</b>"
            
            print(f"📸 Saved: {category} #{collector.counts[category]}")
            return True
            
        except Exception as e:
            status_widget.value = f"<b style='color: red'>❌ Error: {e}</b>"
            return False
    
    # Button handlers
    def create_button_handler(category):
        def handler(b):
            save_image(category)
        return handler
    
    for category, button in buttons.items():
        button.on_click(create_button_handler(category))
    
    def update_camera_feed():
        """Update camera display"""
        while collector.collecting:
            try:
                img = camera.value
                if img is not None:
                    # Add overlay showing current category
                    img_display = img.copy()
                    current_cat = category_widget.value
                    
                    # Add text overlay
                    font = cv2.FONT_HERSHEY_SIMPLEX
                    cv2.putText(img_display, f"Current: {current_cat}", (10, 30), 
                               font, 0.6, (255, 255, 255), 2)
                    cv2.putText(img_display, collector.categories[current_cat], (10, 60), 
                               font, 0.4, (255, 255, 255), 1)
                    
                    # Resize for display
                    img_display = cv2.resize(img_display, (400, 400))
                    
                    # Convert and display
                    _, buffer = cv2.imencode('.jpg', img_display)
                    image_widget.value = buffer.tobytes()
            except Exception as e:
                print(f"Display error: {e}")
            time.sleep(0.1)
    
    def start_collection(b):
        """Start data collection"""
        collector.collecting = True
        start_button.disabled = True
        stop_button.disabled = False
        
        # Enable all buttons
        for button in buttons.values():
            button.disabled = False
        
        status_widget.value = "<b style='color: blue'>📷 Collection active - Position car and click scenario buttons!</b>"
        
        # Start camera feed
        threading.Thread(target=update_camera_feed, daemon=True).start()
    
    def stop_collection(b):
        """Stop data collection"""
        collector.collecting = False
        start_button.disabled = False
        stop_button.disabled = True
        
        # Disable all buttons
        for button in buttons.values():
            button.disabled = True
        
        # Final count
        total = sum(collector.counts.values())
        status_widget.value = f"<b style='color: orange'>⏹️ Collection stopped - Total: {total} images</b>"
    
    start_button.on_click(start_collection)
    stop_button.on_click(stop_collection)
    
    # Layout
    button_grid = ipywidgets.GridBox([
        buttons['straight_center'], buttons['straight_left_correct'], 
        buttons['straight_right_correct'], buttons['left_turn_gentle'],
        buttons['left_turn_sharp'], buttons['left_turn_exit'],
        buttons['recovery_left'], buttons['recovery_right']
    ], layout=ipywidgets.Layout(grid_template_columns="repeat(2, 180px)"))
    
    count_grid = ipywidgets.VBox([
        ipywidgets.HBox([count_displays['straight_center'], count_displays['straight_left_correct']]),
        ipywidgets.HBox([count_displays['straight_right_correct'], count_displays['left_turn_gentle']]),
        ipywidgets.HBox([count_displays['left_turn_sharp'], count_displays['left_turn_exit']]),
        ipywidgets.HBox([count_displays['recovery_left'], count_displays['recovery_right']])
    ])
    
    controls = ipywidgets.VBox([
        ipywidgets.HBox([start_button, stop_button]),
        ipywidgets.HTML("<h4>📊 Image Counts:</h4>"),
        count_grid,
        ipywidgets.HTML("<h4>🎯 Quick Capture:</h4>"),
        button_grid,
        status_widget
    ])
    
    interface = ipywidgets.VBox([
        instructions,
        ipywidgets.HBox([image_widget, controls])
    ])
    
    # Initialize counts
    update_counts()
    
    print("✅ Improved collection interface ready!")
    print("\n🎯 COLLECTION TARGETS:")
    print("Each category: 30-50 images")
    print("Total target: 300-400 images")
    print("Focus on RECOVERY scenarios to fix oversteer!")
    
    return interface

def collection_strategy_guide():
    """Guide for collecting better training data"""
    print("\n📋 IMPROVED COLLECTION STRATEGY")
    print("=" * 40)
    
    print("🎯 PRIORITY COLLECTION ORDER:")
    print("1. Recovery scenarios (30-40 each) - FIXES OVERSTEER!")
    print("   • recovery_left: Car too far left → steer right")
    print("   • recovery_right: Car too far right → steer left")
    print()
    print("2. Straight corrections (30-40 each)")
    print("   • straight_left_correct: Slightly left → center")
    print("   • straight_right_correct: Slightly right → center")
    print()
    print("3. Perfect straights (40-50)")
    print("   • straight_center: Perfect center-line driving")
    print()
    print("4. Left turns (30-40 each)")
    print("   • left_turn_gentle: Easy curves")
    print("   • left_turn_sharp: Tight turns")
    print("   • left_turn_exit: Straightening after turn")
    
    print(f"\n🔧 HOW THIS FIXES YOUR ISSUES:")
    print("• Better straights: Multiple straight scenarios")
    print("• Better left turns: Entry, apex, exit examples") 
    print("• Fixes oversteer: Recovery scenarios teach correction")
    print("• More robust: Handles all driving situations")

# Main execution
def start_improved_data_collection():
    """Start the improved data collection process"""
    print("🚀 STARTING IMPROVED DATA COLLECTION")
    print("=" * 45)
    
    # Clear old data
    clear_old_training_data()
    
    # Show strategy
    collection_strategy_guide()
    
    # Create interface
    interface = create_improved_collection_interface()
    
    if interface:
        print("\n📸 IMPROVED DATA COLLECTION READY!")
        print("Focus on RECOVERY scenarios to fix oversteer issue!")
        display(interface)
    else:
        print("❌ Failed to create collection interface")

# Ready to use
print("\n📸 IMPROVED DATA COLLECTION READY!")
print("=" * 40)
print("🎯 This will collect better training data:")
print("  • Specific driving scenarios")
print("  • Recovery examples (fixes oversteer)")
print("  • Better straights and turns")
print("  • More robust training data")
print()
print("🚀 Run: start_improved_data_collection()")

📸 IMPROVED DATA COLLECTION FOR BETTER TRAINING

📸 IMPROVED DATA COLLECTION READY!
🎯 This will collect better training data:
  • Specific driving scenarios
  • Recovery examples (fixes oversteer)
  • Better straights and turns
  • More robust training data

🚀 Run: start_improved_data_collection()


In [5]:
start_improved_data_collection()

🚀 STARTING IMPROVED DATA COLLECTION
🗑️ CLEARING OLD TRAINING DATA
✅ Old data cleared - ready for fresh collection

📋 IMPROVED COLLECTION STRATEGY
🎯 PRIORITY COLLECTION ORDER:
1. Recovery scenarios (30-40 each) - FIXES OVERSTEER!
   • recovery_left: Car too far left → steer right
   • recovery_right: Car too far right → steer left

2. Straight corrections (30-40 each)
   • straight_left_correct: Slightly left → center
   • straight_right_correct: Slightly right → center

3. Perfect straights (40-50)
   • straight_center: Perfect center-line driving

4. Left turns (30-40 each)
   • left_turn_gentle: Easy curves
   • left_turn_sharp: Tight turns
   • left_turn_exit: Straightening after turn

🔧 HOW THIS FIXES YOUR ISSUES:
• Better straights: Multiple straight scenarios
• Better left turns: Entry, apex, exit examples
• Fixes oversteer: Recovery scenarios teach correction
• More robust: Handles all driving situations

🎛️ CREATING IMPROVED COLLECTION INTERFACE

📷 Setting up camera...
❌ Camera

### Model

The following widget can be used to label a multi-class x, y dataset.  It supports labeling only one instance of each class per image (ie: only one dog), but multiple classes (ie: dog, cat, horse) per image are possible.

Click the image on the top left to save an image of ``category`` to ``dataset`` at the clicked location.

| Widget | Description |
|--------|-------------|
| dataset | Selects the active dataset |
| category | Selects the active category |
| epochs | Sets the number of epochs to train for |
| train | Trains on the active dataset for the number of epochs specified |
| evaluate | Evaluates the accuracy on the active dataset over one epoch |
| model path | Sets the active model path |
| load | Loads a model from the active model path |
| save | Saves a model to the active model path |
| stop | Disables the live demo |
| live | Enables the live demo |

In [2]:
# 🏁 COMPLETE AUTONOMOUS DRIVING CODE
# All fixes applied: Direction fixed + Oversteer fixed

import torch
import torchvision
import torchvision.transforms as transforms
import cv2
import time
import numpy as np

print("🏁 COMPLETE AUTONOMOUS DRIVING SYSTEM")
print("=" * 45)

# FORCE CPU (matching your training)
torch.cuda.is_available = lambda: False
device = torch.device('cpu')

# STABLE RACING PARAMETERS (Fixed oversteer issue)
class StableRacingParams:
    THROTTLE = 0.25          # Stable speed
    STEERING_GAIN = 0.6      # FIXED: Reduced from 1.2 to prevent oversteer
    STEERING_BIAS = 0.0      # No additional bias needed
    MAX_STEERING = 0.8       # Limit maximum steering
    MIN_STEERING = -0.8      # Limit minimum steering  
    UPDATE_RATE = 0.05       # 20Hz updates

def load_trained_model():
    """Load your excellent trained model"""
    print("🤖 Loading trained model...")
    
    # Create model architecture (must match training)
    model = torchvision.models.resnet18(weights=None)
    model.fc = torch.nn.Linear(512, 2)  # x, y coordinates
    model = model.to(device)
    
    try:
        # Load your trained weights
        model.load_state_dict(torch.load('road_following_category_model_cpu.pth', map_location='cpu'))
        model.eval()
        print("✅ Model loaded successfully!")
        print("📊 Training loss: 0.0056 (Excellent!)")
        return model
    except Exception as e:
        print(f"❌ Error loading model: {e}")
        return None

def initialize_camera():
    """Initialize camera - ONE TIME ONLY"""
    print("📷 Initializing camera...")
    try:
        from jetcam.csi_camera import CSICamera
        camera = CSICamera(width=224, height=224, capture_fps=21)
        camera.running = True
        print("✅ Camera initialized")
        time.sleep(3)  # Give camera time to warm up
        return camera
    except Exception as e:
        print(f"❌ Camera failed: {e}")
        return None

def initialize_car():
    """Initialize your calibrated car"""
    print("🚗 Initializing car...")
    
    import traitlets
    import smbus
    
    class Racecar(traitlets.HasTraits):
        steering = traitlets.Float()
        throttle = traitlets.Float()
        
        @traitlets.validate('steering')
        def _clip_steering(self, proposal):
            return max(-1.0, min(1.0, proposal['value']))
            
        @traitlets.validate('throttle')
        def _clip_throttle(self, proposal):
            return max(-1.0, min(1.0, proposal['value']))

    class NvidiaRacecar(Racecar):
        i2c_address = traitlets.Integer(default_value=0x40)
        steering_gain = traitlets.Float(default_value=-0.65)
        steering_offset = traitlets.Float(default_value=0.17)  # Your calibrated offset
        steering_channel = traitlets.Integer(default_value=0)
        throttle_gain = traitlets.Float(default_value=0.8)
        throttle_channel = traitlets.Integer(default_value=1)
        
        def __init__(self, *args, **kwargs):
            super(NvidiaRacecar, self).__init__(*args, **kwargs)
            self.bus = smbus.SMBus(7)
            self.center_pulse = 1500
            self._initialize_pca9685()
            print(f"✅ Car ready with steering offset: {self.steering_offset}")
        
        def _initialize_pca9685(self):
            try:
                self.bus.write_byte_data(self.i2c_address, 0x00, 0x10)
                time.sleep(0.005)
                prescale = int(25000000 / (4096 * 50) - 1)
                self.bus.write_byte_data(self.i2c_address, 0xFE, prescale)
                self.bus.write_byte_data(self.i2c_address, 0x00, 0x20)
                time.sleep(0.005)
                self.bus.write_byte_data(self.i2c_address, 0x01, 0x04)
                self._set_servo_pulse(0, self.center_pulse)
                self._set_servo_pulse(1, self.center_pulse)
            except Exception as e:
                print(f"Warning: {e}")
        
        def _set_servo_pulse(self, channel, pulse_us):
            pulse_us = max(1000, min(2000, pulse_us))
            pwm_value = int((pulse_us * 4096) / 20000)
            base_reg = 0x06 + 4 * channel
            try:
                self.bus.write_byte_data(self.i2c_address, base_reg, 0)
                self.bus.write_byte_data(self.i2c_address, base_reg + 1, 0)
                self.bus.write_byte_data(self.i2c_address, base_reg + 2, pwm_value & 0xFF)
                self.bus.write_byte_data(self.i2c_address, base_reg + 3, (pwm_value >> 8) & 0xFF)
            except Exception as e:
                print(f"PWM Error: {e}")
        
        @traitlets.observe('steering')
        def _on_steering(self, change):
            scaled = change['new'] * self.steering_gain + self.steering_offset
            pulse_us = self.center_pulse + (scaled * 500)
            self._set_servo_pulse(self.steering_channel, pulse_us)
        
        @traitlets.observe('throttle')
        def _on_throttle(self, change):
            scaled = change['new'] * self.throttle_gain
            if scaled > 0:
                pulse_us = self.center_pulse - (scaled * 200)
            elif scaled < 0:
                pulse_us = self.center_pulse + (abs(scaled) * 200)
            else:
                pulse_us = self.center_pulse
            self._set_servo_pulse(self.throttle_channel, pulse_us)
    
    try:
        car = NvidiaRacecar()
        return car
    except Exception as e:
        print(f"❌ Car initialization failed: {e}")
        return None

def preprocess_image(image):
    """Preprocess image for your trained model"""
    transform = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
    
    try:
        # Convert BGR to RGB
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        # Apply transforms and add batch dimension
        image_tensor = transform(image_rgb).unsqueeze(0).to(device)
        return image_tensor
    except Exception as e:
        print(f"Preprocessing error: {e}")
        return None

def setup_autonomous_system():
    """Setup complete autonomous driving system"""
    print("\n🚀 SETTING UP AUTONOMOUS SYSTEM")
    print("=" * 40)
    
    # Load all components
    model = load_trained_model()
    if model is None:
        print("❌ Cannot proceed without model")
        return None, None, None
    
    camera = initialize_camera()
    if camera is None:
        print("❌ Cannot proceed without camera")
        return None, None, None
    
    car = initialize_car()
    if car is None:
        print("❌ Cannot proceed without car")
        return None, None, None
    
    print("\n✅ AUTONOMOUS SYSTEM READY!")
    print("🔧 ALL FIXES APPLIED:")
    print(f"   ✅ Steering direction: Fixed (inverted)")
    print(f"   ✅ Oversteer issue: Fixed (gain reduced to {StableRacingParams.STEERING_GAIN})")
    print(f"   ✅ Throttle: {StableRacingParams.THROTTLE} (stable speed)")
    print(f"   ✅ Model: Loss 0.0056 (excellent)")
    print(f"   ✅ Steering offset: 0.17 (calibrated)")
    
    return model, camera, car

def start_autonomous_driving(model, camera, car):
    """Start stable autonomous driving with all fixes"""
    print("\n🏁 STARTING AUTONOMOUS DRIVING!")
    print("=" * 40)
    
    print("🔧 STABLE DRIVING PARAMETERS:")
    print(f"   Throttle: {StableRacingParams.THROTTLE}")
    print(f"   Steering gain: {StableRacingParams.STEERING_GAIN} (reduced to prevent oversteer)")
    print(f"   Max steering: ±{StableRacingParams.MAX_STEERING}")
    print(f"   Update rate: {1/StableRacingParams.UPDATE_RATE:.0f} Hz")
    
    print("\n⚠️ SAFETY CHECK:")
    print("• Manual override ready?")
    print("• Clear track with good lighting?")
    print("• Track has LEFT turns (car will turn left)?")
    
    input("\nPress ENTER when ready to start autonomous driving...")
    
    print("\n⏱️ Starting in 3 seconds...")
    for i in range(3, 0, -1):
        print(f"   {i}...")
        time.sleep(1)
    
    print("🚀 AUTONOMOUS DRIVING ACTIVE!")
    
    # Set stable throttle
    car.throttle = StableRacingParams.THROTTLE
    
    frame_count = 0
    start_time = time.time()
    steering_history = []
    
    try:
        print("🎯 Driving with stable parameters - should complete laps smoothly...")
        
        while True:
            # Get camera image
            image = camera.value
            if image is None:
                continue
            
            # AI prediction
            processed_image = preprocess_image(image)
            if processed_image is not None:
                with torch.no_grad():
                    prediction = model(processed_image)
                    raw_steering = float(prediction[0][0])
                
                # APPLY ALL FIXES:
                # Fix 1: Invert steering direction (for left-turn track)
                steering_prediction = -raw_steering
                
                # Fix 2: Reduce gain to prevent oversteer
                ai_steering = steering_prediction * StableRacingParams.STEERING_GAIN
                
                # Fix 3: Apply steering limits
                ai_steering = max(StableRacingParams.MIN_STEERING, 
                                min(StableRacingParams.MAX_STEERING, ai_steering))
                
                # Send to car (car applies your 0.17 offset automatically)
                car.steering = ai_steering
                
                # Track steering for analysis
                steering_history.append(ai_steering)
                frame_count += 1
                
                # Regular progress updates
                if frame_count % 25 == 0:
                    elapsed = time.time() - start_time
                    fps = frame_count / elapsed
                    
                    # Determine direction
                    if ai_steering < -0.1:
                        direction = "LEFT"
                    elif ai_steering > 0.1:
                        direction = "RIGHT"
                    else:
                        direction = "STRAIGHT"
                    
                    # Calculate average steering over last 25 frames
                    recent_avg = np.mean(steering_history[-25:]) if len(steering_history) >= 25 else np.mean(steering_history)
                    
                    print(f"🏁 Frame {frame_count} | FPS: {fps:.1f} | "
                          f"Raw: {raw_steering:+.2f} | AI: {ai_steering:+.2f} | "
                          f"{direction} | Avg: {recent_avg:+.2f}")
            
            time.sleep(StableRacingParams.UPDATE_RATE)
            
    except KeyboardInterrupt:
        print("\n🛑 AUTONOMOUS DRIVING STOPPED BY USER")
        
    except Exception as e:
        print(f"\n❌ Error during autonomous driving: {e}")
        import traceback
        traceback.print_exc()
        
    finally:
        # Safe stop sequence
        print("\n🛑 STOPPING SAFELY...")
        try:
            car.throttle = 0.0
            car.steering = 0.0
            print("✅ Car stopped safely")
        except:
            print("⚠️ Error stopping car - use manual override!")
        
        # Final statistics
        if frame_count > 0:
            elapsed = time.time() - start_time
            avg_fps = frame_count / elapsed
            
            print(f"\n📊 AUTONOMOUS DRIVING STATISTICS:")
            print(f"   Duration: {elapsed:.1f} seconds")
            print(f"   Frames processed: {frame_count}")
            print(f"   Average FPS: {avg_fps:.1f}")
            
            if steering_history:
                avg_steering = np.mean(steering_history)
                steering_range = max(steering_history) - min(steering_history)
                left_turns = len([s for s in steering_history if s < -0.1])
                straight_driving = len([s for s in steering_history if -0.1 <= s <= 0.1])
                right_turns = len([s for s in steering_history if s > 0.1])
                
                print(f"   Average steering: {avg_steering:+.3f}")
                print(f"   Steering range: {steering_range:.3f}")
                print(f"   Driving distribution: {left_turns} left, {straight_driving} straight, {right_turns} right")
        
        print("🏁 Autonomous driving session complete!")

# ULTRA-STABLE MODE (if normal mode still has issues)
class UltraStableParams:
    THROTTLE = 0.2           # Slower
    STEERING_GAIN = 0.4      # Very conservative
    STEERING_BIAS = 0.0
    MAX_STEERING = 0.6       # Very limited
    MIN_STEERING = -0.6
    UPDATE_RATE = 0.05

def start_ultra_stable_driving(model, camera, car):
    """Ultra-stable mode for difficult tracks"""
    print("\n🐌 ULTRA-STABLE AUTONOMOUS DRIVING")
    print("(Very conservative for challenging tracks)")
    print("=" * 45)
    
    car.throttle = UltraStableParams.THROTTLE
    frame_count = 0
    
    try:
        while True:
            image = camera.value
            if image is None:
                continue
            
            processed_image = preprocess_image(image)
            if processed_image is not None:
                with torch.no_grad():
                    prediction = model(processed_image)
                    raw_steering = float(prediction[0][0])
                
                # Ultra-conservative steering
                steering_prediction = -raw_steering  # Invert direction
                ai_steering = steering_prediction * UltraStableParams.STEERING_GAIN  # Very low gain
                
                # Tight limits
                ai_steering = max(UltraStableParams.MIN_STEERING, 
                                min(UltraStableParams.MAX_STEERING, ai_steering))
                
                car.steering = ai_steering
                
                frame_count += 1
                if frame_count % 20 == 0:
                    direction = "LEFT" if ai_steering < -0.05 else "RIGHT" if ai_steering > 0.05 else "STRAIGHT"
                    print(f"🐌 Frame {frame_count}: Raw={raw_steering:+.2f} → AI={ai_steering:+.2f} → {direction}")
            
            time.sleep(UltraStableParams.UPDATE_RATE)
            
    except KeyboardInterrupt:
        print("\n🛑 Ultra-stable driving stopped")
    finally:
        car.throttle = 0.0
        car.steering = 0.0
        print("🛑 Car safely stopped")

# MAIN EXECUTION FUNCTIONS
def autonomous_drive():
    """Main function to start autonomous driving"""
    print("🚀 COMPLETE AUTONOMOUS DRIVING SYSTEM")
    print("All fixes applied - ready for smooth autonomous laps!")
    print()
    
    # Setup system
    model, camera, car = setup_autonomous_system()
    
    if model and camera and car:
        # Start autonomous driving with stable parameters
        start_autonomous_driving(model, camera, car)
    else:
        print("❌ System setup failed - cannot start autonomous driving")

def ultra_stable_drive():
    """Ultra-stable driving for difficult tracks"""
    print("🐌 ULTRA-STABLE AUTONOMOUS DRIVING")
    print("For very challenging tracks or initial testing")
    print()
    
    # Setup system
    model, camera, car = setup_autonomous_system()
    
    if model and camera and car:
        input("Press ENTER to start ultra-stable mode...")
        start_ultra_stable_driving(model, camera, car)
    else:
        print("❌ System setup failed")

# Ready to use!
print("\n🏁 AUTONOMOUS DRIVING READY!")
print("=" * 35)
print("🚀 NORMAL MODE:")
print("   Run: autonomous_drive()")
print("   • Stable parameters")
print("   • Good speed and responsiveness")
print("   • All fixes applied")
print()
print("🐌 ULTRA-STABLE MODE:")
print("   Run: ultra_stable_drive()")
print("   • Very conservative")
print("   • For difficult tracks")
print("   • Extremely stable")
print()
print("✅ All fixes applied:")
print("   • Steering direction: Fixed")
print("   • Oversteer: Fixed") 
print("   • Sensitivity: Tuned")
print("   • Model: Excellent (0.0056 loss)")
print("   • Ready for autonomous laps!")

🏁 COMPLETE AUTONOMOUS DRIVING SYSTEM

🏁 AUTONOMOUS DRIVING READY!
🚀 NORMAL MODE:
   Run: autonomous_drive()
   • Stable parameters
   • Good speed and responsiveness
   • All fixes applied

🐌 ULTRA-STABLE MODE:
   Run: ultra_stable_drive()
   • Very conservative
   • For difficult tracks
   • Extremely stable

✅ All fixes applied:
   • Steering direction: Fixed
   • Oversteer: Fixed
   • Sensitivity: Tuned
   • Model: Excellent (0.0056 loss)
   • Ready for autonomous laps!


In [3]:
autonomous_drive()

🚀 COMPLETE AUTONOMOUS DRIVING SYSTEM
All fixes applied - ready for smooth autonomous laps!


🚀 SETTING UP AUTONOMOUS SYSTEM
🤖 Loading trained model...
❌ Error loading model: [Errno 2] No such file or directory: 'road_following_category_model_cpu.pth'
❌ Cannot proceed without model
❌ System setup failed - cannot start autonomous driving


/tmp/ipykernel_5557/3286801459.py:38: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('road_following_category_model_cpu.pth', map_location='c

In [12]:
# Find your model file
import os

print("🔍 Looking for your model file...")

# Check current directory
current_files = [f for f in os.listdir('.') if f.endswith('.pth')]
print(f"📁 Current directory .pth files: {current_files}")

# Check common locations
common_paths = [
    './road_following_model_cpu.pth',
    './notebooks/road_following_model_cpu.pth', 
    '/home/checker/road_following_model_cpu.pth',
    '/home/checker/Autonomous-Racecar/road_following_model_cpu.pth',
    '/home/checker/notebooks/road_following_model_cpu.pth'
]

print("\n📍 Checking common locations:")
for path in common_paths:
    if os.path.exists(path):
        print(f"✅ FOUND: {path}")
        print(f"   Size: {os.path.getsize(path):,} bytes")
    else:
        print(f"❌ Not found: {path}")

# Search for any .pth files
print("\n🔍 Searching for ALL .pth files...")
for root, dirs, files in os.walk('/home/checker'):
    for file in files:
        if file.endswith('.pth') and 'road_following' in file:
            full_path = os.path.join(root, file)
            size = os.path.getsize(full_path)
            print(f"📄 Found: {full_path} ({size:,} bytes)")

🔍 Looking for your model file...
📁 Current directory .pth files: ['road_following_model_cpu.pth']

📍 Checking common locations:
✅ FOUND: ./road_following_model_cpu.pth
   Size: 44,789,468 bytes
❌ Not found: ./notebooks/road_following_model_cpu.pth
❌ Not found: /home/checker/road_following_model_cpu.pth
❌ Not found: /home/checker/Autonomous-Racecar/road_following_model_cpu.pth
❌ Not found: /home/checker/notebooks/road_following_model_cpu.pth

🔍 Searching for ALL .pth files...
📄 Found: /home/checker/jetracer/notebooks/road_following_model_cpu.pth (44,789,468 bytes)
📄 Found: /home/checker/Autonomous-Racecar/notebooks/road_following_model_cpu.pth (44,789,468 bytes)


In [7]:
# 🔧 CORRECT THROTTLE TEST - Using 1400-1500 Range
import smbus
import time

print("🔧 THROTTLE TEST - Correct Range 1400-1500")
print("=" * 40)

class CorrectThrottleTest:
    def __init__(self):
        self.bus = smbus.SMBus(7)
        self.address = 0x40
        self.throttle_channel = 1
        
        # CORRECT throttle range
        self.center_pulse = 1500
        self.max_forward = 1400   # Forward = lower values
        self.max_reverse = 1500   # Only goes down to center, no reverse?
        
        print(f"🚗 Using CORRECT throttle range:")
        print(f"   Forward: {self.max_forward}μs")
        print(f"   Neutral: {self.center_pulse}μs")
        print(f"   Range: {self.max_forward}-{self.center_pulse}μs")
        
        self.initialize_pca9685()
        print("✅ Correct throttle test ready")
    
    def initialize_pca9685(self):
        try:
            self.bus.write_byte_data(self.address, 0x00, 0x10)
            time.sleep(0.005)
            prescale = int(25000000 / (4096 * 50) - 1)
            self.bus.write_byte_data(self.address, 0xFE, prescale)
            self.bus.write_byte_data(self.address, 0x00, 0x20)
            time.sleep(0.005)
            self.bus.write_byte_data(self.address, 0x01, 0x04)
            self.bus.write_byte_data(self.address, 0xFA, 0x00)
            self.bus.write_byte_data(self.address, 0xFB, 0x00)
            self.bus.write_byte_data(self.address, 0xFC, 0x00)
            self.bus.write_byte_data(self.address, 0xFD, 0x00)
            self.set_servo_position(self.throttle_channel, self.center_pulse)
            print("✅ PCA9685 initialized")
        except Exception as e:
            print(f"❌ Init failed: {e}")
            raise
    
    def set_servo_position(self, channel, pulse_us):
        pulse_us = max(1000, min(2000, pulse_us))
        pwm_val = int((pulse_us * 4096) / 20000)
        base_reg = 0x06 + 4 * channel
        
        try:
            self.bus.write_byte_data(self.address, base_reg, 0)
            self.bus.write_byte_data(self.address, base_reg + 1, 0)
            self.bus.write_byte_data(self.address, base_reg + 2, pwm_val & 0xFF)
            self.bus.write_byte_data(self.address, base_reg + 3, (pwm_val >> 8) & 0xFF)
            print(f"Set throttle to {pulse_us}μs")
        except Exception as e:
            print(f"❌ Servo error: {e}")
    
    def test_correct_range(self):
        """Test the correct 1400-1500 range"""
        print("\n🚗 TESTING CORRECT RANGE: 1400-1500")
        
        # Start at neutral
        print("1. Neutral (1500μs)...")
        self.set_servo_position(self.throttle_channel, 1500)
        time.sleep(2)
        
        # Test different forward speeds in the correct range
        forward_speeds = [1480, 1460, 1440, 1420, 1400]
        
        for speed in forward_speeds:
            print(f"2. Forward speed ({speed}μs)...")
            self.set_servo_position(self.throttle_channel, speed)
            time.sleep(3)
            
            print("   Back to neutral...")
            self.set_servo_position(self.throttle_channel, 1500)
            time.sleep(1)
        
        print("✅ Correct range test complete!")

# Test the correct range
try:
    throttle_test = CorrectThrottleTest()
    throttle_test.test_correct_range()
except Exception as e:
    print(f"❌ Test failed: {e}")

🔧 THROTTLE TEST - Correct Range 1400-1500
🚗 Using CORRECT throttle range:
   Forward: 1400μs
   Neutral: 1500μs
   Range: 1400-1500μs
Set throttle to 1500μs
✅ PCA9685 initialized
✅ Correct throttle test ready

🚗 TESTING CORRECT RANGE: 1400-1500
1. Neutral (1500μs)...
Set throttle to 1500μs
2. Forward speed (1480μs)...
Set throttle to 1480μs
   Back to neutral...
Set throttle to 1500μs
2. Forward speed (1460μs)...
Set throttle to 1460μs
   Back to neutral...
Set throttle to 1500μs
2. Forward speed (1440μs)...
Set throttle to 1440μs
   Back to neutral...
Set throttle to 1500μs
2. Forward speed (1420μs)...
Set throttle to 1420μs
   Back to neutral...
Set throttle to 1500μs
2. Forward speed (1400μs)...
Set throttle to 1400μs
   Back to neutral...
Set throttle to 1500μs
✅ Correct range test complete!
